In [1]:
import os
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [22]:
# Step 1: 读取 Borough 边界数据
# geojson_url = "https://services.arcgis.com/OLiydejKCZTGhvWg/arcgis/rest/services/London_Boroughs/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=geojson"
# boroughs = gpd.read_file(geojson_url)

# 替换为你本地下载后的 geojson 路径
path = "../data/Borough_Boundaries.geojson"

# 正确读取
boroughs = gpd.read_file(path)

# 保留 name 和 geometry 字段
boroughs = boroughs[["name", "gss_code", "geometry"]].rename(columns={"name": "borough"})

print(boroughs)
print(boroughs.columns)

                   borough   gss_code  \
0     Kingston upon Thames  E09000021   
1                  Croydon  E09000008   
2                  Bromley  E09000006   
3                 Hounslow  E09000018   
4                   Ealing  E09000009   
5                 Havering  E09000016   
6               Hillingdon  E09000017   
7                   Harrow  E09000015   
8                    Brent  E09000005   
9                   Barnet  E09000003   
10                 Lambeth  E09000022   
11               Southwark  E09000028   
12                Lewisham  E09000023   
13               Greenwich  E09000011   
14                  Bexley  E09000004   
15                 Enfield  E09000010   
16          Waltham Forest  E09000031   
17               Redbridge  E09000026   
18                  Sutton  E09000029   
19    Richmond upon Thames  E09000027   
20                  Merton  E09000024   
21              Wandsworth  E09000032   
22  Hammersmith and Fulham  E09000013   
23  Kensington a

In [23]:
# Step 2: 设置 OSMnx 配置
ox.settings.log_console = False
ox.settings.use_cache = True

In [24]:
# Step 3: 初始化结果列表
results = []

In [25]:
# Step 4: 遍历每个 Borough，获取道路网络并计算中心性指标
for idx, row in tqdm(boroughs.iterrows(), total=len(boroughs), desc="Processing boroughs"):
    borough_name = row["borough"]
    gss_name = row["gss_code"]
    geometry = row["geometry"]

    try:
        print(f"🚦 Processing: {borough_name}")
        
        # 4.1 获取 borough 范围内的机动车路网
        G = ox.graph_from_polygon(geometry, network_type="drive", simplify=True)

        # 4.2 计算节点中心性
        betweenness = nx.betweenness_centrality(G, weight="length", k=100, seed=42)
        degree = dict(G.degree())
        nx.set_node_attributes(G, betweenness, "betweenness")
        nx.set_node_attributes(G, degree, "degree")

        # 4.3 构造带 u-v 信息的边表
        edge_data = []
        for u, v, key, data in G.edges(keys=True, data=True):
            edge_data.append({
                "u": u,
                "v": v,
                "key": key,
                "geometry": data.get("geometry", None),
                "betweenness": (G.nodes[u]["betweenness"] + G.nodes[v]["betweenness"]) / 2,
                "degree": (G.nodes[u]["degree"] + G.nodes[v]["degree"]) / 2
            })
        edges_df = gpd.GeoDataFrame(edge_data, geometry="geometry", crs="EPSG:4326")

        # 4.4 聚合结果
        summary = {
            "borough": borough_name,
            "gss_code": gss_name,
            "mean_betweenness": edges_df["betweenness"].mean(),
            "max_betweenness": edges_df["betweenness"].max(),
            "mean_degree": edges_df["degree"].mean(),
            "max_degree": edges_df["degree"].max(),
            "edge_count": len(edges_df)
        }
        results.append(summary)

    except Exception as e:
        print(f"❌ Failed for {borough_name}: {e}")
        continue

Processing boroughs:   0%|          | 0/33 [00:00<?, ?it/s]

🚦 Processing: Kingston upon Thames


Processing boroughs:   0%|          | 0/33 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [6]:
# Step 5: 保存结果为 CSV 文件
df_results = pd.DataFrame(results)
df_results.to_csv("london_borough_road_centrality.csv", index=False)
print("All done! Results saved to 'london_borough_road_centrality.csv'")


All done! Results saved to 'london_borough_road_centrality.csv'


In [7]:
print(df_results)

                   borough  mean_betweenness  max_betweenness  mean_degree  \
0     Kingston upon Thames          0.020622         0.261008     5.271409   
1                  Croydon          0.012210         0.177637     5.383993   
2                  Bromley          0.012135         0.172504     5.425875   
3                 Hounslow          0.018356         0.335207     5.247866   
4                   Ealing          0.015021         0.212142     5.418720   
5                 Havering          0.012923         0.179474     5.369130   
6               Hillingdon          0.013128         0.331130     5.215237   
7                   Harrow          0.016811         0.175713     5.383765   
8                    Brent          0.016646         0.352244     5.400981   
9                   Barnet          0.013282         0.203354     5.415900   
10                 Lambeth          0.017742         0.284172     5.446247   
11               Southwark          0.014075         0.219205   